# DEL 3.3 - Bruk av `pydantic` med et enkelt API

I denne delen skal vi bruke et åpent api med enkel datastruktur
til å få litt praktisk erfaring med bruk av `pydantic`


In [ ]:
# Importer bibliotekene vi trenger
import requests
import json
from pydantic import BaseModel, ConfigDict

# For automatiske tester trenger vi pytest og ipytest
import ipytest
import pytest

ipytest.autoconfig()

***

## 3.3.1: REST Countries

API-et fra [https://restcountries.com](https://restcountries.com) kan brukes for
å hente ut en del grunninformasjon om land. Det har en enkel datastruktur,
og er raskt å sette seg inn i samt enkelt å bruke.

> _Før du går videre:_
>
> Gå inn på [restcountries](https://restcountries.com) og bla litt i
> dokumentasjonen. Ha denne lett tilgjengelig når du ser videre på oppgavene
> under.

## 3.3.2: Test en API-forespørsel

Kjør cellen under for å gjøre en enkel forespørsel til API-et. Studer data som
fra responsen, for å gjøre deg kjent med datastrukturen.


In [ ]:
response = requests.get("https://restcountries.com/v3.1/name/norge")
data = response.json()
print(json.dumps(data, indent=2))

## 3.3.3: `Pydantic` modeller til _restcountries_

Vi ser fra responsdata at vi har flere ulike objekt-typer i dataen vi får om et
land. Det gjør det naturlig å lage flere `pydantic`-klasser, som kan samles i
en hovedklasse som representerer en respons fra api-et. Da får vi en nøstet
struktur, slik som tidligere da vi brukte den egendefinerte klassen `Adresse`
som type for egenskap i klassen `Kontakt`.

Det kan også være at vi får mer data i responsen fra api-et enn vi egentlig trenger,
og vi skal se litt på hvordan dette kan håndteres.


In [ ]:
class CountryNameEntry(BaseModel):
    common: str
    official: str


class CountryName(CountryNameEntry):
    nativeName: dict[str, CountryNameEntry]


class Currency(BaseModel):
    name: str
    symbol: str


class Country(BaseModel):
    name: CountryName
    population: int
    area: float
    region: str
    subregion: str
    languages: dict[str, str]
    currencies: dict[str, Currency]


## La oss teste modellene våre med data fra api-et:
def country_model_test():
    response = requests.get("https://restcountries.com/v3.1/name/norge")
    data = response.json()
    country_data = data[0]  # Vi tar det første landet i listen (Norge)
    country = Country(
        **country_data
    )  # Bruker "unpacking" for å sende dataen til modellen
    print("Utskrift av Country objekt:")
    print(country)
    print("\nUtskrift av data i objektet, dumpet i json-format:")
    print(country.model_dump_json(indent=2))


country_model_test()

## Oppgaver (3.3-1 -> 3.3-2)

#### Oppgave 3.3-1

- Hvilken datatype har variabelen `country` (i funksjonen `country_model_test`)?
- Hvilken datatype har `country.population`?
- Hvilken datatype har `country.name.nativeName['nno']`?

> MERK!
>
> Svar med den faktiske datatypen disse data har fått, ikke original datatype fra
> `response` eller dumpet datatype fra `country.model_dump_json`

> Tips:
>
> Du kan opprette en ny _Code_-celle og bruke `type` for å sjekke datatypene


_Skriv ditt svar her_


#### Oppgave 3.3-2

- Hvilke egenskaper har klassen `Currency`?
- Hvilke egenskaper har klassen `CountryName` (OBS: husk arvede egenskaper)?
- Hva skjedde med ekstra data fra responsen vi ikke inkluderte i `Country`?


_Skriv ditt svar her_


## 3.3.4: Håndtere ekstra data

### Ignorer ekstra data (standard)

Over ble ekstra data fra responsen droppet. Dette er standard i `BaseModel`.
Vi kan også håndtere det på andre måter. 

### `ConfigDict`

Vi kan modifisere egenskaper ved når vi lager `pydantic`-modeller ved å
modifisere den spesielle egenskapen `model_config` av typen `ConfigDict`.

For å bruke denne `ConfigDict` må den importeres fra `pydantic`. Dette er
allerede gjort om du har kjørt første kodecelle i notebooken, som inkluderer
følgende linje:

```python
from pydantic import BaseModel, ConfigDict
```


### Ikke tillat ekstra egenskaper
Hvis ønskelig, kan vi velge at det skal skrives ut en feilmelding dersom
en prøver å opprette et nytt `Contry`-objekt med flere egenskaper enn
det som er definert i modellen. La oss redefinere `Country` med denne
konfigurasjonen:


In [ ]:
class Country(BaseModel):
    model_config = ConfigDict(
        extra="forbid"
    )  # Forbyr ekstra felter som ikke er definert i modellen

    name: CountryName
    population: int
    area: float
    region: str
    subregion: str
    languages: dict[str, str]
    currencies: dict[str, Currency]


# Kjør testen igjen for å se hvordan det fungerer med extra="forbid"
country_model_test()

### Dynamisk håndtering av ekstra egenskaper

Vi kan også tillate at ekstra egenskaper legges til dynamisk:


In [ ]:
class Country(BaseModel):
    model_config = ConfigDict(
        extra="allow"
    )  # Inkluder ekstra felter som ikke er definert i modellen

    name: CountryName
    population: int
    area: float
    region: str
    subregion: str
    languages: dict[str, str]
    currencies: dict[str, Currency]


# Kjør testen igjen for å se hvordan det fungerer med extra="allow"
country_model_test()

## Oppgaver (3.3-3 -> 3.3-4)

### Oppgave 3.3-3

Forklar forskjellen på de tre konfigurasjonene vist over,
- standard
- `extra="forbid"`
- `extra="allow"`


_Skriv ditt svar her_


### Oppgave 3.3-4

Nå skal du få utvide definisjonene fra tidligere med flere egenskaper.

- Implementer modell for `PostalCode`
- Implementer modell for en annen egenskap av `Country`
- Legg de nye modellene til i definisjonen av `Country`
- Finn et par a andre egenskaper i responsdata fra apiet med en enkel datatype
  du ikke trenger å lage en egen modell for. Legg til definisjoner i `Country`
  for disse egenskapene.

Bruk startkoden under og fyll inn kode som svarer på oppgaven.


In [ ]:
class CountryNameEntry(BaseModel):
    common: str
    official: str


class CountryName(CountryNameEntry):
    nativeName: dict[str, CountryNameEntry]


class Currency(BaseModel):
    name: str
    symbol: str


## --- FYLL INN DET SOM MANGLER --- ##
class PostalCode(BaseModel):
    pass  # Implementer PostalCode modellen her


# Definer en modell for en annen egenskap som
# ikke er definert ennå
class ValgfriEgenskap(BaseModel):
    # Erstatt med passende klasse-navn

    # Fyll inn egenskaper til modellen du har valgt å definere her
    pass


class Country(BaseModel):
    name: CountryName
    population: int
    area: float
    region: str
    subregion: str
    languages: dict[str, str]
    currencies: dict[str, Currency]
    # Legg til de to nye modellene som egenskaper her
    # Også legg til et par andre egenskaper du finner i respons-
    # dataen fra api-et, som er egenskaper som ikke trenger
    # egne modeller (slik som region og area over)


Kjør cellen under for å sjekke at du har løst oppgaven riktig

In [ ]:
%%ipytest


def test_extended_country_model():
    response = requests.get("https://restcountries.com/v3.1/name/norge")
    data = response.json()
    country_data = data[0]
    country = Country(**country_data)
    postal_code_cls_fields = PostalCode.model_fields.keys()
    assert postal_code_cls_fields == {"format", "regex"}, (
        "PostalCode modellen har ikke de forventede egenskapene"
    )
    fields = country.model_fields_set
    assert "postalCode" in fields, "Fant ikke egenskapen postalCode i Country modellen"
    assert isinstance(country.postalCode, PostalCode), (
        "postalCode er ikke av typen PostalCode"
    )
    assert len(fields) > 10, (
        "Country modellen ser ut til å mangle noen egenskaper. Har du lagt til fire nye egenskaper?"
    )
